In [1]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import airy, airye, spherical_jn as sph_jn, spherical_yn as sph_yn
import os

# dirs = [n for n in os.listdir() if (os.path.isdir(n) and (n[-2:]=='_p'))]
dirs = ['H_0_plus_p']
for dir in dirs:
    r, hls, ddrls = load_matrices(dir)
    print(dir)
    emin = hls[-1,0,0]
    n = len(hls[0])
    shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
    vd = (hls-shift)/4.637

    hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
    emax = np.abs(np.array(hfunc(r[0])[-1,-1]))

    w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
    wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])

    es = np.array([1])
    for i in range(n):
        emax = np.abs(np.array(hfunc(r[0])[i,i]))
        emin = np.abs(np.array(hfunc(r[-1])[i,i]))
        es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
    es = np.unique(np.round(np.sort(es)))
    sigmas = np.zeros((len(es), n, n))
    ecm = es*4.637+np.real(hls[-1,0,0])
    for ei in range(len(es)):
        e = es[ei]
        lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
        ks = np.diag(np.sqrt(e*np.eye(n)-hfunc(r[-1])))
        yf = np.zeros((lmax, n, n), dtype=complex)
        nt = np.sum(np.diag(vd[-1]) < e)
        for l in range(lmax):
            wtest = w(r[:-1]+np.diff(r)/2, l, e)
            wptest = wp(r[:-1]+np.diff(r)/2, l)
            k2 = np.zeros((len(wtest), n))
            t = np.zeros((len(wtest), n, n), dtype=complex)
            wpt = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                k2[i], t[i] = np.linalg.eigh(wtest[i])
                wpt[i] = np.matrix(t[i]).H @ (np.matrix(wptest[i]) @ np.matrix(t[i]))
            p = np.array([np.matrix(t[i]).H @ np.matrix(t[i-1]) if i > 0 else np.matrix(t[0]) for i in range(len(wtest))])
            c1 = np.zeros((len(wtest), n, n), dtype=complex)
            c2 = np.zeros((len(wtest), n, n), dtype=complex)
            c3 = np.zeros((len(wtest), n, n), dtype=complex)
            c4 = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                alpha = np.array([-(wpt[i,j,j])**(1./3.) if np.allclose(-(wpt[i,j,j])**(1./3.), np.real(-(wpt[i,j,j])**(1./3.)))
                                  else -(wpt[i,j,j])**(1./3.)*np.exp(-1j*np.angle(-(wpt[i,j,j])**(1./3.))) for j in range(n)])
                beta = k2[i]/np.diag(wpt[i])
                wr = alpha/np.pi
                rhob = np.diff(r)[i]/2
                x = np.array([[alpha[j]*(-rhob+beta[j]), alpha[j]*(rhob+beta[j])] for j in range(n)])
                for j in range(n):
                    if np.any(np.real(x[j]) > 100):
                        dksi = np.diff((2./3)*(x[j]**(3./2)))[0]
                        aie, aiep, bie, biep = airye(x[j])
                        c1[i,j,j] = np.pi*(np.exp(-dksi)*aie[1]*biep[0]-np.exp(dksi)*bie[1]*aiep[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(np.exp(dksi)*bie[1]*aie[0]-np.exp(-dksi)*aie[1]*bie[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(np.exp(-dksi)*aiep[1]*biep[0]-np.exp(dksi)*biep[1]*aiep[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(np.exp(dksi)*biep[1]*aie[0]-np.exp(-dksi)*aiep[1]*bie[0]) #/wr[j]
                    else:
                        ai, aip, bi, bip = airy(x[j])
                        c1[i,j,j] = np.pi*(ai[1]*bip[0]-bi[1]*aip[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(bi[1]*ai[0]-ai[1]*bi[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(aip[1]*bip[0]-bip[1]*aip[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(bip[1]*ai[0]-aip[1]*bi[0]) #/wr[j]
            y = np.zeros((len(r), n, n), dtype=complex)
            y[0] = 1e308*np.eye(n, dtype=complex)
            for i in range(len(wtest)):
                try:
                    prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
                except np.linalg.LinAlgError:
                    print(np.matrix(c1[i] + c2[i] @ y[i]))
                # print(prey)
                y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)
                # print(y[i+1])
            yf[l] = np.matrix(t[-1]).H @ (y[-1] @ np.matrix(t[-1]))
        jmat = np.array([np.diag([sph_jn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
        nmat = np.array([np.diag([sph_yn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
        jpmat = np.array([np.diag([k*sph_jn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
        npmat = np.array([np.diag([k*sph_yn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
        rmat = np.array([-np.matrix(npmat[l] - yf[l] @ nmat[l]).I @ np.matrix(jpmat[l] - yf[l] @ jmat[l]) for l in range(lmax)])
        smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
        sigmas[ei,:nt,:nt] = np.pi * 1e-16 * np.sum(np.array([(2*l+1)*(np.abs(smat[l]-np.eye(nt))**2) for l in range(lmax)]),axis=0) @ np.matrix(np.abs(np.diag(ks[:nt]))**2).I
        print(ecm[ei])
        print(sigmas[ei])
    headerfinal = 'E, cm-1\t'
    for i in range(n):
        for j in range(n):
            headerfinal += f'Sigma_{i}{j}, cm2\t'
    tofile = np.zeros((len(es), n*n+1))
    tofile[:,0] = ecm
    for i in range(n):
        for j in range(n):
            tofile[:,i*n+j+1] = sigmas[:,i,j]

    np.savetxt(f'{dir}/sigmas_total_airy.txt', tofile, fmt='%.6e', delimiter='\t', header=headerfinal, comments='')
    print(f'Directory {dir} is done!')

H_0_plus_p
105370.147
[[3.34391986e-14 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]
105374.784
[[2.67236342e-14 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]
105379.42099999999
[[2.73790056e-14 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.

/tmp/ipykernel_2686044/2942525072.py:76: RuntimeWarning: overflow encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
/tmp/ipykernel_2686044/2942525072.py:76: RuntimeWarning: invalid value encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)


108578.951
[[1.85039038e-14 6.86245129e-16 2.62872053e-17 2.73976786e-18
  8.96758418e-18]
 [4.63756612e-16 3.57173293e-14 6.38663107e-17 8.56996576e-18
  3.78185551e-18]
 [8.32379247e-18 2.98475516e-17 3.52439334e-14 1.19930471e-15
  1.87047397e-19]
 [7.33533097e-19 3.38995167e-18 1.01663611e-15 3.57322870e-14
  1.29193982e-19]
 [1.51273790e-20 9.49509632e-21 9.92688364e-22 8.12221756e-22
  3.26083001e-14]]
108588.22499999999
[[1.93275257e-14 7.63326165e-16 2.00246978e-17 1.95469649e-18
  1.05009794e-17]
 [5.16492258e-16 3.54597535e-14 7.54682938e-17 1.10419473e-17
  3.73986481e-18]
 [6.36929655e-18 3.54116391e-17 3.30392937e-14 1.25042268e-15
  1.11914162e-19]
 [5.25621725e-19 4.38546634e-18 1.06090072e-15 3.53525116e-14
  1.34023701e-19]
 [2.02073309e-20 1.06668344e-20 6.69764440e-22 9.69927913e-22
  3.09657524e-14]]
108592.862
[[1.87729963e-14 7.25949894e-16 1.86487546e-17 1.95620098e-18
  7.86758012e-18]
 [4.91512794e-16 3.65563339e-14 6.80635604e-17 9.51762023e-18
  2.13985054e-1